In [1]:
import polars as pl

# Adjust so all columns in dataframe are shown
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_width_chars(1000)

polars.config.Config

In [2]:
df = pl.read_parquet("/data/scratch/ankile/surveillance-metric-2/data/pool_swap_events/*.parquet")

f"{df.shape[0]:,} rows, {df.shape[1]} columns"

'16,416,874 rows, 19 columns'

In [12]:
df.select("transaction_hash").head(5).to_pandas()["transaction_hash"].to_list()

['0x8d7fedce6b88bc54f620434c2964363b9feeb71598bfac3e91c9e0044b797758',
 '0xc8d4808c18e522741595c1e2d2366a96c08c51be8a7257d898038f57156104d1',
 '0xb17e059667122af566d3f17a65f76b5f937ae88c26b4e04e9a1dbfe315eb3544',
 '0x450dfb2e06824443054124d49fb24dd20c5bab2b7f7d5635e79a7747960aba57',
 '0x717fa590ae33fac5dc44d17fe1078661660f7a5ba849630cba954c43d42ef08f']

In [4]:
df.columns

['chain_name',
 'address',
 'block_timestamp',
 'block_number',
 'transaction_hash',
 'log_index',
 'sender',
 'recipient',
 'amount0',
 'amount1',
 'sqrtPriceX96',
 'liquidity',
 'tick',
 'to_address',
 'from_address',
 'transaction_index',
 'gas_price',
 'gas_used',
 'l1_fee']

In [13]:
df.filter(pl.col("transaction_hash") == "0x8d7fedce6b88bc54f620434c2964363b9feeb71598bfac3e91c9e0044b797758")

chain_name,address,block_timestamp,block_number,transaction_hash,log_index,sender,recipient,amount0,amount1,sqrtPriceX96,liquidity,tick,to_address,from_address,transaction_index,gas_price,gas_used,l1_fee
str,str,"datetime[μs, UTC]",i64,str,i64,str,str,str,str,str,str,f64,str,str,i64,str,str,null
"""ethereum""","""0x000ea4a83acefdd62b1b43e9ccc2…",2023-01-21 06:47:23 UTC,16453477,"""0x8d7fedce6b88bc54f620434c2964…",216,"""0xef1c6e67703c7bd7107eed8303fb…","""0x3f70347ccc6992b85b71c36f32d4…","""-38.81568558473759688""","""0.023576958766568552""","""1956496701769534125377830665""","""137209476420281066764""",-74028.0,"""0xef1c6e67703c7bd7107eed8303fb…","""0x3f70347ccc6992b85b71c36f32d4…",91,"""15693206651""","""140343""",null


In [3]:
from google.cloud import bigquery
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

# Initialize BigQuery client
client = bigquery.Client()

# Set your BigQuery details
project_id = "mimetic-design-338620"
dataset_id = "uniswap"
table_id = "swap"

# Set the output directory for Parquet files
output_dir = "output_parquet"
os.makedirs(output_dir, exist_ok=True)

# Construct the full table ID
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Function to process and save data in chunks
def process_and_save_chunk(rows, chunk_id):
    df = pd.DataFrame(
        data=[list(row.values()) for row in rows],
        columns=[field.name for field in rows.schema]
    )
    table = pa.Table.from_pandas(df)
    pq.write_table(table, os.path.join(output_dir, f"chunk_{chunk_id}.parquet"))

# Query to select all data from the table
query = f"SELECT * FROM `{full_table_id}`"

# Set up the query job with large result set configuration
job_config = bigquery.QueryJobConfig(allow_large_results=True)
query_job = client.query(query, job_config=job_config)

# Process the results in chunks
chunk_size = 100000  # Adjust based on your memory constraints
chunk_id = 0

while True:
    chunk = query_job.result(start_index=chunk_id * chunk_size, max_results=chunk_size)
    rows = list(chunk)
    
    if not rows:
        break
    
    process_and_save_chunk(chunk, chunk_id)
    chunk_id += 1
    print(f"Processed and saved chunk {chunk_id}")

print("Download complete. All data saved as Parquet files.")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.